In [3]:
import numpy as np
import pandas as pd
import math 
import matplotlib.pyplot as plt
from scipy import spatial
import pickle
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
with open('/content/drive/MyDrive/NLP_proj/words.pkl', 'rb') as f:
    words = pickle.load(f)

with open('/content/drive/MyDrive/NLP_proj/words_vectors.pkl', 'rb') as f:
    x = np.asarray(pickle.load(f))

In [12]:
n = len(words) 

d = np.zeros((n,n))
d_cur = np.zeros((n,n))

# calculate distance matrix

In [ ]:
for i in range (0,n):
    if i % 1000 == 0:
        print(f'{i}/{n}')
    for j in range (i,n):
        d[i][j] = spatial.distance.cosine(x[i], x[j])
        d[j][i] = d[i][j]

# finde keys

In [ ]:
def detect_key(d_cur , c_cur):
    m = d_cur.shape[0]
    temp = []
    k = []
    s = []
    n = 0
#############   find I1    ############# min mean

    for i in range (0,m):# i1
        sum = 0
        for j in range (0,m):
            sum += d_cur[i][j]

        temp.append(sum / m) 

    i1 = temp.index(sorted(temp)[0])  


    for t in range (0,m):# update s and k for i1
        if t != i1:
            k.append(t)
    s.append(i1)

########################################



    while( n < c_cur-1 ):
        min_i = []
#############   find In    ############# max min

        for i in k: #finde min 
            min = 8000
            for j in s:
                if d_cur[i][j] < min:
                    min = d_cur[i][j]
            min_i.append(min)# list of min values in each row

        p = 0
        for i in k:# find max of mins
            for j in s:
                if d_cur[i][j] == max( min_i):
                    i_n = i
                    k.remove(i_n)
                    p = 1
                    break
            if p == 1:
                break

        s.append(i_n)

        # unique_numbers = set(s)
        # ss = []

        # for number in unique_numbers:
        #     ss.append(number)
        # s = ss
        n+=1
    return s

     



# main code

In [ ]:
def key_clustering( dd , c_target, k1 , g ):


########################################### initial

    n = dd.shape[0]
    k = k1+1
    end_key = 2
    list_k=[]
    r = np.zeros((n,k))

############################################ calculate R
    for i in range (0,n):
        list_k = sorted(dd[i])[0:k] # take 2 min value k = 2 
        for j in range(0,n):  # finde index of that vslues
            for x in range(0,len(list_k)):
                if dd[i][j] == list_k[x] :
                        r[i][x] = j

                                         
    l = list(range(n))
    c_per = n
    c_cur = math.floor(n/g)
    s_cur = []
    ########################### calculate d_cur

    for i in range (0,n):
        for j in range(0,n):
            temp = 0
            for a in range (0,k):
                for b in range (0,k):
                    temp = temp + dd[int(r[i][a] ),int(r[j][b])]
            d_cur[i][j] = temp / ((k)*(k))
    
    ############## loop #############

    while c_cur > c_target :
            s_cur = detect_key(d_cur ,c_cur)

            ############## update labels ##############
            for i in range(len(l)):
                    min = 80000 # initialize with infinity
                    if(l[i] not in s_cur):
                        for j in s_cur:
                            if d_cur[l[i],j] < min:
                                min = d_cur[ l[i] , j ]
                                index = j
                        l[i] = index
    
            #################  update distance ###############
            unique_values =list( Counter(l).keys()) # equals to list(set(words))
            unique_count = list(Counter(l).values()) # counts the elements' frequency
            samp = []

            for i in unique_count:
                e = np.zeros((i*(3)))
                samp.append(e)
            
            p = []
            
            for x in range(0,len(unique_values)):
                nn = 0
                for y in range(0,len(l)):
                    if unique_values[x] == l[y]:
                        samp[x][nn] = y
                        nn = nn+1

                        for i in range(1,k):
                            samp[x][nn] = r[y][i] 
                            nn = nn+1

            sample = []
            lo = []
            for e in samp:
                lo.append(list(map(int, e))) 
            
            label_dict = dict(zip(unique_values, lo))

            for x,y  in label_dict.items():
                for p,q in label_dict.items():
                    sum  = 0
                    for a in y:
                        for b in q:
                            sum+=dd[a,b]

                    d_cur[x,p] = sum /(len(q)*len(y))

            c_per = c_cur
            c_cur = math.floor(c_cur/g)

    s_final = detect_key(d_cur ,c_target)

    ############## update labels ##############
    for i in range(len(l)):
        min = 80000
        if(l[i] not in s_final):
            for j in s_final:
                if d_cur[l[i],j] < min:
                    min = d_cur[ l[i] , j ]
                    index = j
            l[i] = index

    return l

In [ ]:
dd = np.copy(d)
c_target = 5
k1 = 2
g = 3
key_clustering(dd, c_target, k1 , g)